In [ ]:
import json
import warnings
from importlib import reload

import maite.protocols.image_classification as ic
import pandas as pd
import test_stages.daml_test_stage
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.models import ResNet50_Weights, resnet50
from utils import collect_metrics, collect_report_consumables, load_models_and_datasets, run_stages

reload(test_stages.daml_test_stage)

from test_stages.daml_test_stage import DamlTestStage

warnings.filterwarnings("ignore")

# Configure Pipeline Stages


### Panel Inputs

Based directly off of Panel mockup. Values expected to change


In [ ]:
# ['Accuracy', 'mAP']
metric = "Accuracy"

# Float
performance = 0.92

Load model


In [ ]:
# Mock model
weights = ResNet50_Weights.DEFAULT
preprocess = weights.transforms()
model = resnet50(weights)
isinstance(model, ic.Model)  # Not sure where Maite Protocol check is

# Mock model with different weights
comparison_weights = ResNet50_Weights.IMAGENET1K_V1
comparison_model = resnet50(comparison_weights)
isinstance(comparison_model, ic.Model)  # Not sure where in Maite Protocol check is

Load datasets


In [ ]:
# FMOW was too large for testing
class MaiteMNIST(Dataset):
    def __init__(self, train=True, transforms=None):
        self.dataset = MNIST("../data/", train=train, transform=transforms, download=True)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        imgs, labels = self.dataset[idx]
        return imgs, labels, {}


# ResNet50 wants 3 channels, MNIST is 1
t = transforms.Compose([transforms.Grayscale(num_output_channels=3), preprocess])
dev_dataset = MaiteMNIST(True, t)
op_dataset = MaiteMNIST(False, t)
print(f"Dev Dataset is MAITE compliant: {isinstance(dev_dataset, ic.Dataset)}")
print(f"Op Dataset is MAITE compliant: {isinstance(op_dataset, ic.Dataset)}")

In [ ]:
sample_config_json = '{"dev": ["image properties", "visual quality", "duplicates", "outliers", "balance", "coverage", "parity", "feasibility", "sufficiency"], "op": ["image properties", "visual quality", "duplicates", "outliers", "balance", "coverage", "parity", "drift", "out-of-distribution"]}'
# sample_config_json = '{"dev": ["image properties", "duplicates", "outliers", "balance", "coverage", "feasibility", "sufficiency"], "op": ["image properties", "visual quality", "outliers", "balance", "parity", "drift", "out-of-distribution"]}'
config_json = json.loads(sample_config_json)

Create Stage


In [ ]:
ds = DamlTestStage(config_json=config_json)
stages = [ds]

# Pipeline


In [ ]:
load_models_and_datasets(
    dev_dataset=dev_dataset,
    op_dataset=op_dataset,
    model=model,
    comparison_model=comparison_model,
    target_performance=performance,
    stages=stages,
)

In [ ]:
run_stages(stages=stages)

In [ ]:
df = collect_metrics(stages=stages)
print(df)
pd.Series(df)

In [ ]:
collect_report_consumables(stages=stages)